In [26]:
pip install tensorflow-addons

In [27]:
# Adapted KERAS tutorial
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, AlphaDropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow_addons.layers import WeightNormalization

In [28]:
import os
import pickle
import numpy as np

In [29]:
batch_size = 32
num_classes = 10
epochs = 20
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_weight_trained_model.h5'

In [30]:
# list devices so you can check whether your gpu is available
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [31]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [32]:
model = Sequential()

model.add(WeightNormalization(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:],kernel_initializer='lecun_normal',bias_initializer='zeros')))
model.add(Activation('relu'))
model.add(WeightNormalization(Conv2D(32, (3, 3),kernel_initializer='lecun_normal',bias_initializer='zeros')))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(AlphaDropout(0.1))

model.add(WeightNormalization(Conv2D(64, (3, 3), padding='same',kernel_initializer='lecun_normal',bias_initializer='zeros')))
model.add(Activation('relu'))
model.add(WeightNormalization(Conv2D(64, (3, 3),kernel_initializer='lecun_normal',bias_initializer='zeros')))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(AlphaDropout(0.1))

model.add(Flatten())
model.add(Dense(512,kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(Activation('relu'))
model.add(AlphaDropout(0.2))
model.add(Dense(num_classes,kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(Activation('softmax'))

In [33]:
# initiate RMSprop optimizer
opt = keras.optimizers.legacy.RMSprop(learning_rate=0.0001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [34]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [35]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))

Using real-time data augmentation.
Epoch 1/20
1562/1562 [==============================] - 45s 27ms/step - loss: 1.8815 - accuracy: 0.3268 - val_loss: 1.7700 - val_accuracy: 0.4533
Epoch 2/20
1562/1562 [==============================] - 37s 23ms/step - loss: 1.5689 - accuracy: 0.4373 - val_loss: 1.8919 - val_accuracy: 0.4941
Epoch 3/20
1562/1562 [==============================] - 37s 23ms/step - loss: 1.4285 - accuracy: 0.4896 - val_loss: 1.5405 - val_accuracy: 0.5637
Epoch 4/20
1562/1562 [==============================] - 36s 23ms/step - loss: 1.3251 - accuracy: 0.5271 - val_loss: 1.4415 - val_accuracy: 0.5922
Epoch 5/20
1562/1562 [==============================] - 39s 25ms/step - loss: 1.2511 - accuracy: 0.5550 - val_loss: 1.5252 - val_accuracy: 0.6010
Epoch 6/20
1562/1562 [==============================] - 42s 27ms/step - loss: 1.1885 - accuracy: 0.5787 - val_loss: 1.3644 - val_accuracy: 0.6290
Epoch 7/20
1562/1562 [==============================] - 36s 23ms/step - loss: 1.1372 - ac

In [36]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/saved_models/keras_cifar10_weight_trained_model.h5 


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)

print('Model Accuracy = %.5f' % (evaluation[1]))

f = open('CIFAR10_SELU_results.txt', 'a')
f.write(' Test accuracy:' + str(evaluation[1]) +  '\n')
f.close()


predict_gen = model.predict(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)


312/312 [==============================] - 8s 25ms/step - loss: 0.9703 - accuracy: 0.7281
Model Accuracy = 0.72806
312/312 [==============================] - 13s 37ms/step
